## **Exercício 2**

#### Exercício 5.13: Incerteza quântica no oscilador harmônico

Em unidades onde todas as constantes são 1, a função de onda do n-ésimo nível de energia do oscilador harmônico quântico unidimensional — ou seja, uma partícula puntiforme sem spin em um poço de potencial quadrático — é dada por

$$\psi_n(x) = \frac{1}{\sqrt{2^n n! \sqrt{\pi}}} e^{-x^2/2} H_n(x)$$

para $n = 0... \infty$, onde $H_n(x)$ é o n-ésimo polinômio de Hermite. Os polinômios de Hermite satisfazem uma relação um tanto similar à dos números de Fibonacci, embora mais complexa:

$$H_{n+1}(x) = 2xH_n(x) - 2nH_{n-1}(x)$$

Os dois primeiros polinômios de Hermite são $H_0(x) = 1$ e $H_1(x) = 2x$.

a. Escreva uma função definida pelo usuário `H(n,x)` que calcula $H_n(x)$ para um dado $x$ e qualquer inteiro $n \ge 0$. Use sua função para fazer um gráfico que mostre as funções de onda do oscilador harmônico para $n = 0, 1, 2$ e $3$, todas no mesmo gráfico, no intervalo de $x = -4$ a $x = 4$. Dica: Existe uma função `factorial` no pacote `math` que calcula o fatorial de um inteiro.

b. Faça um gráfico separado da função de onda para $n = 30$ de $x = -10$ a $x = 10$. Dica: Se o seu programa demorar muito para rodar neste caso, então você está fazendo o cálculo de forma errada — o programa deve levar apenas um segundo ou mais para rodar.

c. A incerteza quântica na posição de uma partícula no n-ésimo nível de um oscilador harmônico pode ser quantificada pela sua raiz da posição quadrática média $\sqrt{\langle x^2 \rangle}$, onde

$$\langle x^2 \rangle = \int_{-\infty}^{\infty} x^2 |\psi_n(x)|^2 dx$$

Escreva um programa que avalie essa integral usando a quadratura Gaussiana com 100 pontos, e então calcule a incerteza (ou seja, a raiz da posição quadrática média da partícula) para um dado valor de $n$. Use seu programa para calcular a incerteza para $n = 5$. Você deve obter uma resposta na vizinhança de $\sqrt{\langle x^2 \rangle} = 2.3$.